# Projekt MSP1 / 2024
Cílem tohoto projektu je se seznámit s programovými nástroji využívaných ve statistice a osvojit si základní procedury. Projekt není primárně zaměřen na efektivitu využívání programového vybavení (i když úplně nevhodné konstrukce mohou mít vliv na hodnocení), ale nejvíce nás zajímají vaše statistické závěry a způsob vyhodnocení. Dbejte také na to, že každý graf musí splňovat nějaké podmínky - přehlednost, čitelnost, popisky.

V projektu budete analyzovat časy běhu šesti různých konfigurací algoritmů. Ke každé konfiguraci vzniklo celkem 200 nezávislých běhů, jejichž logy máte k dispozici v souboru [logfiles.zip](logfiles.zip).

Pokud nemáte rozchozené prostředí pro pro spouštění Jupyter notebooku, můžete využití službu [Google Colab](https://colab.google/). Jakákoliv spolupráce, sdílení řešení a podobně je zakázána!

S případnými dotazy se obracejte na Vojtěcha Mrázka (mrazek@fit.vutbr.cz).

__Odevzdání:__ tento soubor (není potřeba aby obsahoval výstupy skriptů) do neděle 27. 10. 2024 v IS VUT. Kontrola bude probíhat na Pythonu 3.12.3 (standardní instalace Ubuntu); neočekává se však to, že byste používali nějaké speciality a nekompatibilní knihovny. V případě nesouladu verzí a podobných problémů budete mít možnost reklamace a prokázání správnosti funkce. Bez vyplnění vašich komentářů a závěrů do označených buněk nebude projekt hodnocen!

__Upozornění:__ nepřidávejte do notebooku další buňky, odpovídejte tam, kam se ptáme (textové komentáře do Markdown buněk)

__Tip:__ před odevzdáním resetujte celý notebook a zkuste jej spustit od začátku. Zamezíte tak chybám krokování a editací, kdy výsledek z buňky na konci použijete na začátku.

__OTÁZKA K DOPLNĚNÍ:__

Timotej Halenár

xhalen00

## Načtení potřebných knihoven
Načtěte knihovny, které jsou nutné pro zpracování souborů a práci se statistickými funkcemi.

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import json
from zipfile import ZipFile

## Načtení dat do DataFrame
Ze souboru `logfiles.zip` umístěném ve stejném adresáři načtěte data a vytvořte Pandas DataFrame.

Výsledky jsou uložené ve formátu JSON - pro zpracování použijte knihovnu `json`.
Můžete využít následující kostru - je vhodné pracovat přímo se ZIP souborem. Jedinou nevýhodou může být to, že vám bude vracet _byte_ objekt, který musíte přes funkci `decode` zpracovat.

Upravte také pomocí funkce `.astype()` datové typy patřičných sloupců.

```py
data = []
with ZipFile("logfiles.zip") as zf:
    for filename in zf.namelist():
        # TODO test názvu souboru
        with zf.open(filename, "r") as f:
            pass # vytvořte slovník

df = pd.DataFrame(data)
df
```

In [ ]:
data = []
with ZipFile("logfiles.zip") as zf:
    for filename in zf.namelist():
        if filename.endswith('.json'):
            with zf.open(filename, "r") as f:
                content = f.read().decode('utf-8')
                json_data = json.loads(content)
                data.append(json_data)

df = pd.DataFrame(data)
df

## Analýza a čištění dat
Vhodným způsobem pro všechny konfigurace analyzujte časy běhů a pokud tam jsou, identifikujte hodnoty, které jsou chybné. Vyberte vhodný graf, který zobrazí samostatně jednotlivé konfigurace.

In [ ]:
q_low = df["runtime"].quantile(0.01)
q_hi  = df["runtime"].quantile(0.99)

outliers = df[(df['runtime'] >= q_hi) | (df['runtime'] <= q_low)]

print("outliers count: ", outliers.shape[0])

#Na boxplote sú najvhodnejšie zobrazení outliers
sns.boxplot(x='configuration', y='runtime', data=df)

__OTÁZKA K DOPLNĚNÍ:__

_Objevily se nějaké chybné hodnoty? Proč tam jsou s ohledem na to, že se jedná o běhy algoritmů? Proč jste zvolili tento typ grafu?_

- ak algoritmus zlyhal, môže mať čas behu 0
- príp. ak algoritmus beží príliš dlho, dostane sa na maximálnu hodnotu (TIMEOUT) 1800
- na boxplote je jasne vidieť outliers a dá sa ľahko porovnať jedna konfigurácia s druhou


Vyčistěte dataframe `df` tak, aby tam tyto hodnoty nebyly a ukažte znovu analýzu toho, že čištění dat bylo úspěšné. Odtud dále pracujte s vyčištěným datasetem.

In [ ]:
df = df[(df['runtime'] < q_hi) & (df['runtime'] > q_low)]

sns.boxplot(x='configuration', y='runtime', data=df)

## Deskriptivní popis hodnot
Vypište pro jednotlivé konfigurace základní deskriptivní parametry.  

__TIP__ pokud výsledky uložíte jako Pandas DataFrame, zobrazí se v tabulce.

In [ ]:
df_1 = df[df['configuration'] == "config1"]
df_2 = df[df['configuration'] == "config2"]
df_3 = df[df['configuration'] == "config3"]
df_4 = df[df['configuration'] == "config4"]
df_5 = df[df['configuration'] == "config5"]
df_6 = df[df['configuration'] == "config6"]

def get_stats(frame):
    stats = frame["runtime"].describe()
    print(stats[['mean', 'std', 'min', 'max']])
    print("Run count:", frame.shape[0])
    #print(stats)

print("*********************************")
print("Config 1")
get_stats(df_1)
print("*********************************")
print("Config 2")
get_stats(df_2)
print("*********************************")
print("Config 3")
get_stats(df_3)
print("*********************************")
print("Config 4")
get_stats(df_4)
print("*********************************")
print("Config 5")
get_stats(df_5)
print("*********************************")
print("Config 6")
get_stats(df_6)
print("*********************************")


__OTÁZKA K DOPLNĚNÍ:__

_Okomentujte, co všechno můžeme z parametrů vyčíst._

Konfigurácie 1 a 6 sú na základe strednej hodnoty najviac porovnateľné, ale konfigurácia 6 má značne vyšší rozptyl, a teda je menej konzistentná. Naopak pri konfigurácii 1 je rozptyl nízky, časy všetkých behov sú bližšie pri sebe. Pri týchto dvoch konfiguráciach nemôžme jasne určiť, ktorá je rýchlejšia. Najrýchlejší beh konfigurácie 6 je rýchlejší, ako pri konfigurácii 1, zároveň však aj naopak najpomalší beh konfigurácie 6 je pomalší ako všetky behy 1. konfigurácie.

## Vizualizace
Vizualizujte časy běhů algoritmů tak, aby byl v jednom grafu zřejmý i rozptyl hodnot, avšak bylo možné porovnání. Zvolte vhodný graf, který pak níže komentujte.

In [ ]:
sns.violinplot(x='configuration', y='runtime', data=df)

__OTÁZKA K DOPLNĚNÍ:__

_Okomentujte  výsledky z tabulky._

Na violinplote je najlepšie vidno rozptyl: pri vysokom rozptyle je graf úzky a natiahnutý, naopak pri nízky rozptyl je graf tučnejší. Konfigurácie 4 a 6 majú najvyšší rozptyl, a teda sú najmenej konzistentné. očividne najmenší rozptyl má konfigurácia 1.

## Určení efektivity konfigurací algoritmů
Nás ale zajímá, jaká konfigurace je nejrychlejší. Z výše vykresleného grafu můžeme vyloučit některé konfigurace. Existuje tam však minimálně jedna dvojice, u které nedokážeme jednoznačně určit, která je lepší - pokud nebudeme porovnávat pouze extrémní hodnoty, které mohou být dané náhodou, ale celkově. Proto proveďte vhodný test významnosti - v následující části diskutujte zejména rozložení dat (i s odkazem na předchozí buňky, variabilitu vs polohu a podobně). Je nutné každý logický krok a výběry statistických funkcí komentovat. 

Vužijte vhodnou funkci z knihovny `scipy.stats` a funkci poté __implementujte sami__ na základě základních matematických funkcí knihovny `numpy` případně i funkcí pro výpočet vybraného rozložení v [scipy.stats](https://docs.scipy.org/doc/scipy/reference/stats.html). Při vlastní implementaci není nutné se primárně soustředit na efektivitu výpočtu (není potřeba využít všechny funkce numpy, můžete použít normální cykly a podobně - v hodnocení však bude zahrnuta přehlednost a neměly by se objevit jasné chyby, jako je zvýšení třídy složitosti a podobně).

__OTÁZKA K DOPLNĚNÍ:__

_Jaká data budete zkoumat? Jaké mají rozložení a parametry (např. varianci) a jaký test použijete? Jaká je nulová hypotéza? Jak se liší variabilita a poloha vybraných konfigurací?_

Konfigurácie 1 a 6 vyplývajú z grafu ako jediní kandidáti na najrýchlejšiu konfiguráciu. Z grafu vyššie vieme určiť, že majú približne normálne rozloženie, a poznáme ich strednú hodnotu a odchýlku. Vykonáme t-test pomocou funkcie `stats.ttest_ind` s prepínačom `equal_var=False`, pretože ich rozptyly sú zásadne rozdielne. Tým uskutočníme Welchov t-test.

H0: Konfigurácie 1 a 6 majú rovnaké priemerné (očakávané) hodnoty


H1: Konfigurácie 1 a 6 majú rozdielne priemerné (očakávané) hodnoty

In [ ]:
conf_1 = df[df['configuration'] == 'config1']['runtime']
conf_6 = df[df['configuration'] == 'config6']['runtime']

res = stats.normaltest(conf_1)

t_stat, t_val = stats.ttest_ind(conf_1, conf_6, equal_var=False)


print(f"t-statistic: {t_stat}, p-value: {t_val}")

__OTÁZKA K DOPLNĚNÍ:__

_Jaký je závěr statistického testu?_

keďže p-hodnota je veľmi malá, zamietame nulovú hypotézu, ktorá tvrdí, že konfigurácie 1 a 6 majú rovnaké očakávané hodnoty. Z výsledku testu vyplýva, že rozdiel v stredných hodnotách konfigurácií 1 a 6 je značný. Môžeme na základe nižšej strednej hodnoty konfigurácie č. 1 uznať, že je rýchlejšia.

### Vlastní implementace
Implementujte stejný test pomocí knihovních funkcí a ukažte, že je výsledek stejný.

In [ ]:
a = np.mean(conf_1)- np.mean(conf_6)

var_6 = np.std(conf_6, ddof=1)**2
var_1 = np.std(conf_1, ddof=1)**2

b = np.sqrt(var_6/conf_6.shape + var_1/conf_1.shape)

# t-statistika
stat = a/b

# stupne volnosti
deg = conf_1.shape[0] + conf_6.shape[0] - 2

crit = stats.t.ppf(0.95, deg)

#p-hodnota
p = (1 - stats.t.cdf(np.abs(stat), deg)) * 2

print(stat, p)
